In [410]:
import glob
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import string
import langid
from langdetect import detect, DetectorFactory
import sys
import numpy as np
from scipy.stats import randint
import scipy
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import chi2, f_regression , mutual_info_classif
from sklearn import metrics
import pandas as pd
%matplotlib inline

path = 'DataSet' # use your path
all_files = glob.glob(path + "/*.xlsx")
li = []

for filename in all_files:
    df = pd.read_excel(filename)
    li.append(df)
frame = pd.concat(li, axis=0, ignore_index=True)
frame = frame.drop(frame[frame['Classification'].isnull()].index)
frame.dropna(subset=['Reviews/Comments'], inplace=True)
frame  = frame.sort_values('Feedback ID')
frame.to_excel("Input.xlsx")


In [411]:
frame

,OYO ID,Hotel ID,OYO Product,Feedback ID,Booking ID,Source,Rating,L1,L2,Reviews/Comments,...,Comfort,Direction,Location,Customer Support,Breakfast,Food,Payment,Pricing,Default,Classification
1062,MRT042,46039.0,SMART,60144582.0,118672380.0,Android App,5.0,NaN,NaN,I had a great stay. Hotel staff was very suppo...,...,True,False,False,False,False,False,False,False,False,Good
283,KOL1340,82429.0,SMART,60156829.0,118141552.0,Android App,1.0,NaN,NaN,"Worst ever OYO experienced. No OYO sign board,...",...,True,True,True,False,False,False,False,False,False,Bad
737,ASR293,58291.0,SMART,60168982.0,118586097.0,bulk_upload,0.0,NaN,NaN,One of the best properties we have ever been i...,...,False,False,False,False,False,False,False,False,True,Good
1465,GRG1010,40843.0,SMART,60178229.0,118844212.0,Android App,5.0,NaN,NaN,Everything is good no complaints staff behavio...,...,False,False,False,False,False,False,False,False,False,Good
654,DEL775,15322.0,SMART,60179150.0,118417540.0,Android App,5.0,NaN,NaN,One of the best hotel through OYO. I recommend...,...,False,False,False,False,False,False,False,False,False,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,GZB108,46852.0,SMART,61382951.0,121800300.0,IOS App,3.0,NaN,NaN,The curtains were very thin and incapable to k...,...,True,False,False,False,False,False,False,False,False,Bad
842,SRG102,56126.0,SMART,61390504.0,117518389.0,Android App,3.0,NaN,NaN,No local ids cards accepted and unmarried coup...,...,False,False,False,False,False,False,False,False,True,Bad
2199,DEL1593,57984.0,Townhouse,61390678.0,122345895.0,Crs,1.0,NaN,NaN,Wash room was not clean \n room was not clean....,...,False,False,False,False,False,False,False,False,False,Bad
1134,KOL1040,57161.0,Silverkey,61392769.0,121787462.0,Android App,1.0,NaN,NaN,The manager with tall hieght told to leave the...,...,False,False,False,False,False,False,False,False,False,Discard


In [412]:
%%time
df = pd.read_excel("Input.xlsx")
unique_sentiments_count = df.groupby('Classification')['Feedback ID'].nunique()
print(unique_sentiments_count)

Classification
Average     194
Bad        1153
Discard     179
Good        862
Name: Feedback ID, dtype: int64
CPU times: user 724 ms, sys: 0 ns, total: 724 ms
Wall time: 723 ms


In [413]:
'''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
import string
import pandas as pd
import re
import string
import langid

def clean_text_round1(text,punctuation):
    text = text.lower()
    text = re.sub('[%s]' % re.escape(punctuation), ' ', text)
    text = re.sub('\n',' ',text)
    text = " ".join(text.split())
    return text

punctuation = string.punctuation.replace("'","")
punctuation = punctuation + "0123456789"
df['comment(1cleaning)'] = df['Reviews/Comments'].apply(lambda x: clean_text_round1(x,punctuation))
df['langid'] = df['Reviews/Comments'].apply(lambda x: langid.classify(x)[0])


In [414]:
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import nltk


stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
            "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
            'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
            'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
            'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
            'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
            'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
            'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 
            'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 
            "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', "aren't", 'couldn', 
            "couldn't", "hadn't", "hasn't", "haven't", "isn't", 'ma', "mightn't", "mustn't", "needn't" , "shan't", 'shouldn', 'wasn', "weren't"]


print(stop)
dir(nltk.corpus)

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

    
def clean_text_round2(text):
    # print(text)
    text = text.split(" ")
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    # print(text)
    pos_tags = pos_tag(text)
    #print(pos_tags)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)     
df['lemmatized_comment'] = df['comment(1cleaning)'].apply(lambda x: clean_text_round2(x))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

In [415]:
df = df.drop(df[df['Classification'].isnull()].index)
df = df.drop(df[df['langid'] !='en'].index)
df.dropna(subset=['lemmatized_comment'], inplace=True)
unique_sentiments_count = df.groupby('Classification')['Feedback ID'].nunique()
print(unique_sentiments_count)

Classification
Average     193
Bad        1150
Discard     172
Good        857
Name: Feedback ID, dtype: int64


In [416]:
def getClass(text):
    if(text == 'Good'):
        return 1
    if(text == 'Bad'):
        return 0
    return 2

df['Class'] = df['Classification'].apply(lambda x: getClass(x))

In [417]:
df

,Unnamed: 0,OYO ID,Hotel ID,OYO Product,Feedback ID,Booking ID,Source,Rating,L1,L2,...,Breakfast,Food,Payment,Pricing,Default,Classification,comment(1cleaning),langid,lemmatized_comment,Class
0,1062,MRT042,46039,SMART,60144582,118672380,Android App,5,NaN,NaN,...,False,False,False,False,False,Good,i had a great stay hotel staff was very suppor...,en,great stay hotel staff supportive kind hearted...,1
1,283,KOL1340,82429,SMART,60156829,118141552,Android App,1,NaN,NaN,...,False,False,False,False,False,Bad,worst ever oyo experienced no oyo sign board m...,en,worst ever oyo experienced no oyo sign board m...,0
2,737,ASR293,58291,SMART,60168982,118586097,bulk_upload,0,NaN,NaN,...,False,False,False,False,True,Good,one of the best properties we have ever been i...,en,one best property ever would like visit strong...,1
3,1465,GRG1010,40843,SMART,60178229,118844212,Android App,5,NaN,NaN,...,False,False,False,False,False,Good,everything is good no complaints staff behavio...,en,everything good no complaint staff behaviour p...,1
4,654,DEL775,15322,SMART,60179150,118417540,Android App,5,NaN,NaN,...,False,False,False,False,False,Good,one of the best hotel through oyo i recommend ...,en,one best hotel oyo recommend hotel guest room ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,989,GZB108,46852,SMART,61382951,121800300,IOS App,3,NaN,NaN,...,False,False,False,False,False,Bad,the curtains were very thin and incapable to k...,en,curtain thin incapable keep sunlight away cann...,0
2385,842,SRG102,56126,SMART,61390504,117518389,Android App,3,NaN,NaN,...,False,False,False,False,True,Bad,no local ids cards accepted and unmarried coup...,en,no local id card accept unmarried couple not w...,0
2386,2199,DEL1593,57984,Townhouse,61390678,122345895,Crs,1,NaN,NaN,...,False,False,False,False,False,Bad,wash room was not clean room was not clean pro...,en,wash room not clean room not clean properly co...,0
2387,1134,KOL1040,57161,Silverkey,61392769,121787462,Android App,1,NaN,NaN,...,False,False,False,False,False,Discard,the manager with tall hieght told to leave the...,en,manager tall hieght tell leave hotel complain ...,2


In [420]:
df = df[(df.Class==1) | (df.Class==0)]

In [421]:
df

,Unnamed: 0,OYO ID,Hotel ID,OYO Product,Feedback ID,Booking ID,Source,Rating,L1,L2,...,Breakfast,Food,Payment,Pricing,Default,Classification,comment(1cleaning),langid,lemmatized_comment,Class
0,1062,MRT042,46039,SMART,60144582,118672380,Android App,5,NaN,NaN,...,False,False,False,False,False,Good,i had a great stay hotel staff was very suppor...,en,great stay hotel staff supportive kind hearted...,1
1,283,KOL1340,82429,SMART,60156829,118141552,Android App,1,NaN,NaN,...,False,False,False,False,False,Bad,worst ever oyo experienced no oyo sign board m...,en,worst ever oyo experienced no oyo sign board m...,0
2,737,ASR293,58291,SMART,60168982,118586097,bulk_upload,0,NaN,NaN,...,False,False,False,False,True,Good,one of the best properties we have ever been i...,en,one best property ever would like visit strong...,1
3,1465,GRG1010,40843,SMART,60178229,118844212,Android App,5,NaN,NaN,...,False,False,False,False,False,Good,everything is good no complaints staff behavio...,en,everything good no complaint staff behaviour p...,1
4,654,DEL775,15322,SMART,60179150,118417540,Android App,5,NaN,NaN,...,False,False,False,False,False,Good,one of the best hotel through oyo i recommend ...,en,one best hotel oyo recommend hotel guest room ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383,905,DEL1525,55641,SPOT ON,61373230,118489815,Integrated OTA,2,NaN,NaN,...,False,False,False,False,True,Bad,refused to provide a discount for booking i ca...,en,refuse provide discount book say anything posi...,0
2384,989,GZB108,46852,SMART,61382951,121800300,IOS App,3,NaN,NaN,...,False,False,False,False,False,Bad,the curtains were very thin and incapable to k...,en,curtain thin incapable keep sunlight away cann...,0
2385,842,SRG102,56126,SMART,61390504,117518389,Android App,3,NaN,NaN,...,False,False,False,False,True,Bad,no local ids cards accepted and unmarried coup...,en,no local id card accept unmarried couple not w...,0
2386,2199,DEL1593,57984,Townhouse,61390678,122345895,Crs,1,NaN,NaN,...,False,False,False,False,False,Bad,wash room was not clean room was not clean pro...,en,wash room not clean room not clean properly co...,0


In [422]:
resampled_df = df
X = resampled_df['lemmatized_comment']
y = resampled_df['Class']

In [205]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(X,y, 
                                                               X.index, test_size=0.25, 
                                                               random_state=42,stratify=y)

In [206]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=3,
                        ngram_range=(1, 2))

#print(df.comment3lemmatize)
# We transform each complaint into a vector
fitted_vectorizer = tfidf.fit(X_train)

features = fitted_vectorizer.transform(X_train)

labels = y_train
print("Each of the %d comment is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

Each of the 1779 comment is represented by 1994 features (TF-IDF score of unigrams and bigrams)


In [207]:
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2,250)
fitted_selector = selector.fit(features,y_train)
features_new = fitted_selector.transform(features)

In [208]:
mask = fitted_selector.get_support() #list of booleans
new_features_names = [] # The list of your K best features
feature_names= tfidf.get_feature_names()
for bool, feature in zip(mask, feature_names):
    if bool:
        new_features_names.append(feature)

In [85]:
# Hyperparamete Tuning
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from hyperopt import hp, tpe
import xgboost as xgb
from hyperopt.fmin import fmin

auc_scorer = make_scorer(roc_auc_score)

def objective(params):
   
    params = {'n_estimators': int(params['n_estimators']),
              'max_depth': int(params['max_depth']),
              'min_child_weight': int(params['min_child_weight']),
              'learning_rate' : float(params['learning_rate']),
              'gamma': float(params['gamma']),
              'colsample_bytree': float(params['colsample_bytree']),
              'subsample': float(params['subsample']),
              'colsample_bylevel': float(params['colsample_bylevel'])
#               ,'class_weight':str(params['class_weight'])
             }
   
    clf = xgb.XGBClassifier(random_state=42, **params)
    score = cross_val_score(clf,features_new,labels, scoring=auc_scorer, cv=StratifiedKFold(n_splits=5)).mean()
    print("AUC {:.3f} params {}".format(score, params))
    return -score

space = {
    'learning_rate':    hp.uniform('learning_rate', 0.01, 0.5),
    'max_depth':        hp.choice('max_depth',        np.arange(1, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'gamma' : hp.uniform('gamma', 0.1, 1.5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.9),
    'subsample':        hp.uniform('subsample', 0.7, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.3, 0.8),
    'n_estimators':     hp.choice('n_estimators', np.arange(10, 1100, 50, dtype=int)),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100)
for i in best.keys():
    best[i] = int(best[i])

best_model = xgb.XGBClassifier(random_state=42, **best)


AUC 0.835 params {'n_estimators': 560, 'max_depth': 2, 'min_child_weight': 4, 'learning_rate': 0.33730459250060224, 'gamma': 1.4506989750818944, 'colsample_bytree': 0.6844639545892011, 'subsample': 0.7899429261573497, 'colsample_bylevel': 0.43539866858425197}
AUC 0.854 params {'n_estimators': 260, 'max_depth': 9, 'min_child_weight': 1, 'learning_rate': 0.2775651937347629, 'gamma': 0.7417703183023474, 'colsample_bytree': 0.779467048644354, 'subsample': 0.716929483914224, 'colsample_bylevel': 0.588329974455629}
AUC 0.854 params {'n_estimators': 460, 'max_depth': 5, 'min_child_weight': 2, 'learning_rate': 0.21259318544940017, 'gamma': 1.4813809945882346, 'colsample_bytree': 0.7836009951570224, 'subsample': 0.8802655641681099, 'colsample_bylevel': 0.7633374505535013}
AUC 0.814 params {'n_estimators': 510, 'max_depth': 3, 'min_child_weight': 6, 'learning_rate': 0.2631197051414458, 'gamma': 0.9286646454071584, 'colsample_bytree': 0.4899990568264899, 'subsample': 0.7745487980903359, 'colsampl

In [209]:
from sklearn.feature_extraction.text import TfidfVectorizer
train, test = train_test_split(df, random_state=42, test_size=0.30, shuffle=True)
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,2), norm='l2')
vectorizer.fit(train.lemmatized_comment)
vectorizer.fit(test.lemmatized_comment)
x_train = vectorizer.transform(train.lemmatized_comment)
y_train = train[['Class']]
x_test = vectorizer.transform(test.lemmatized_comment)
y_test = test[['Class']]

In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import label_ranking_average_precision_score


xgb_model =  xgb.XGBClassifier(objective='multi:softprob', num_class=2, n_estimators= 510, max_depth= 80, min_child_weight= 1, learning_rate= 0.09520194824175505, gamma= 1.1020474587964464,
                                      colsample_bytree= 0.4908567846524142, subsample= 0.7622694310500184, colsample_bylevel= 0.5593257295147386,
                                          scale_pos_weight=train.Class[train.Class==0].count()/train.Class[train.Class==1].count())
            
xgb_model.fit(x_train, y_train)

/usr/local/lib64/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib64/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.5593257295147386,
              colsample_bytree=0.4908567846524142, gamma=1.1020474587964464,
              learning_rate=0.09520194824175505, max_delta_step=0, max_depth=80,
              min_child_weight=1, missing=None, n_estimators=510, n_jobs=1,
              nthread=None, num_class=2, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1.7545605306799337, seed=None, silent=True,
              subsample=0.7622694310500184)

In [127]:
# make predictions for test data
y_pred = xgb_model.predict(x_test)
predictions = [round(value) for value in y_pred]


In [128]:

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(metrics.classification_report(y_test, y_pred))

Accuracy: 88.76%
              precision    recall  f1-score   support

           0       0.89      0.94      0.92       457
           1       0.89      0.79      0.83       255

    accuracy                           0.89       712
   macro avg       0.89      0.87      0.87       712
weighted avg       0.89      0.89      0.89       712



In [227]:
#Adding Booking.com reviews

df1 = pd.read_csv('Hotel_Reviews.csv') 

In [228]:
df1_positive = df1[['Positive_Review']]

In [229]:
df1_negative = df1[['Negative_Review']]

In [230]:
df1_negative = df1_negative[df1_negative.Negative_Review!='No Negative']

In [231]:
df1_negative = df1_negative[df1_negative.Negative_Review!='Nothing']

In [232]:
df1_negative

,Negative_Review
0,I am so angry that i made this post available...
2,Rooms are nice but for elderly a bit difficul...
3,My room was dirty and I was afraid to walk ba...
4,You When I booked with your company on line y...
5,Backyard of the hotel is total mess shouldn t...
...,...
515731,No parking Public parking garage is 15 Euro p...
515733,no trolly or staff to help you take the lugga...
515734,The hotel looks like 3 but surely not 4
515735,The ac was useless It was a hot week in vienn...


In [233]:
df1_positive = df1_positive[df1_positive.Positive_Review!='No Positive']

In [234]:
df1_positive = df1_positive[df1_positive.Positive_Review!='Nothing']

In [235]:
df1_positive

,Positive_Review
0,Only the park outside of the hotel was beauti...
1,No real complaints the hotel was great great ...
2,Location was good and staff were ok It is cut...
3,Great location in nice surroundings the bar a...
4,Amazing location and building Romantic setting
...,...
515732,helpful staff allowed me to check in early as...
515733,location
515734,Breakfast was ok and we got earlier check in
515736,The rooms are enormous and really comfortable...


In [236]:
df1_positive = df1_positive[df1_positive['Positive_Review'].apply(lambda x: len(x.split(' ')) > 3)]

In [237]:
df1_positive['Class'] = 1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [238]:
df1_positive.columns = ['review', 'Class']

In [239]:
df1_negative =  df1_negative[df1_negative['Negative_Review'].apply(lambda x: len(x.split(' ')) > 3)]

In [240]:
df1_negative['Class'] = 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [241]:
df1_negative.columns = ['review', 'Class']

In [242]:
df1_negative

,review,Class
0,I am so angry that i made this post available...,0
2,Rooms are nice but for elderly a bit difficul...,0
3,My room was dirty and I was afraid to walk ba...,0
4,You When I booked with your company on line y...,0
5,Backyard of the hotel is total mess shouldn t...,0
...,...,...
515731,No parking Public parking garage is 15 Euro p...,0
515733,no trolly or staff to help you take the lugga...,0
515734,The hotel looks like 3 but surely not 4,0
515735,The ac was useless It was a hot week in vienn...,0


In [243]:
df1 = df1_positive.append(df1_negative)

In [244]:
df1

,review,Class
0,Only the park outside of the hotel was beauti...,1
1,No real complaints the hotel was great great ...,1
2,Location was good and staff were ok It is cut...,1
3,Great location in nice surroundings the bar a...,1
4,Amazing location and building Romantic setting,1
...,...,...
515731,No parking Public parking garage is 15 Euro p...,0
515733,no trolly or staff to help you take the lugga...,0
515734,The hotel looks like 3 but surely not 4,0
515735,The ac was useless It was a hot week in vienn...,0


In [246]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
            "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
            'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
            'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
            'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
            'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
            'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
            'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 
            'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 
            "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', "aren't", 'couldn', 
            "couldn't", "hadn't", "hasn't", 
            "haven't", "isn't", 'ma', "mightn't", "mustn't", "needn't" , "shan't", 'shouldn', 'wasn', "weren't"]


    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
df1["lemmatized_comment"] = df1["review"].apply(lambda x: clean_text(x))

In [428]:
df1[['lemmatized_comment','Class']]

,lemmatized_comment,Class
0,park outside hotel beautiful,1
1,no real complaint hotel great great location s...,1
2,location good staff ok cute hotel breakfast ra...,1
3,great location nice surroundings bar restauran...,1
4,amaze location building romantic setting,1
...,...,...
515731,no park public parking garage euro per day,0
515733,no trolly staff help take luggage room,0
515734,hotel look like surely not,0
515735,ac useless hot week vienna give hot air,0


In [248]:
from sklearn.feature_extraction.text import TfidfVectorizer
train, test = train_test_split(df1, random_state=42, test_size=0.30, shuffle=True)
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,2), norm='l2')
vectorizer.fit(train.lemmatized_comment)
vectorizer.fit(test.lemmatized_comment)
x_train = vectorizer.transform(train.lemmatized_comment)
y_train = train[['Class']]
x_test = vectorizer.transform(test.lemmatized_comment)
y_test = test[['Class']]

In [ ]:
# Hyperparamete Tuning
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from hyperopt import hp, tpe
import xgboost as xgb
from hyperopt.fmin import fmin

auc_scorer = make_scorer(roc_auc_score)

def objective(params):
   
    params = {'n_estimators': int(params['n_estimators']),
              'max_depth': int(params['max_depth']),
              'min_child_weight': int(params['min_child_weight']),
              'learning_rate' : float(params['learning_rate']),
              'gamma': float(params['gamma']),
              'colsample_bytree': float(params['colsample_bytree']),
              'subsample': float(params['subsample']),
              'colsample_bylevel': float(params['colsample_bylevel'])
#               ,'class_weight':str(params['class_weight'])
             }
   
    clf = xgb.XGBClassifier(random_state=42, **params)
    score = cross_val_score(clf,features_new,labels, scoring=auc_scorer, cv=StratifiedKFold(n_splits=5)).mean()
    print("AUC {:.3f} params {}".format(score, params))
    return -score

space = {
    'learning_rate':    hp.uniform('learning_rate', 0.01, 0.5),
    'max_depth':        hp.choice('max_depth',        np.arange(1, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'gamma' : hp.uniform('gamma', 0.1, 1.5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.9),
    'subsample':        hp.uniform('subsample', 0.7, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.3, 0.8),
    'n_estimators':     hp.choice('n_estimators', np.arange(10, 1100, 50, dtype=int)),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100)
for i in best.keys():
    best[i] = int(best[i])

best_model = xgb.XGBClassifier(random_state=42, **best)

In [249]:

xgb_model =  xgb.XGBClassifier(objective='multi:softprob', num_class=2, n_estimators= 510, max_depth= 80, min_child_weight= 1, learning_rate= 0.09520194824175505, gamma= 1.1020474587964464,
                                      colsample_bytree= 0.4908567846524142, subsample= 0.7622694310500184, colsample_bylevel= 0.5593257295147386)
                                          
            
xgb_model.fit(x_train, y_train)

/usr/local/lib64/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib64/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.5593257295147386,
              colsample_bytree=0.4908567846524142, gamma=1.1020474587964464,
              learning_rate=0.09520194824175505, max_delta_step=0, max_depth=80,
              min_child_weight=1, missing=None, n_estimators=510, n_jobs=1,
              nthread=None, num_class=2, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=0.7622694310500184)

In [187]:
from sklearn.metrics import roc_auc_score

# make predictions for test data
y_pred = xgb_model.predict(x_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(metrics.classification_report(y_test, y_pred))
print("ROC_AUC score: %.2f%%" % (roc_auc_score(y_test, y_pred)))

Accuracy: 95.19%
              precision    recall  f1-score   support

           0       0.96      0.95      0.96    130679
           1       0.94      0.95      0.95    103736

    accuracy                           0.95    234415
   macro avg       0.95      0.95      0.95    234415
weighted avg       0.95      0.95      0.95    234415

ROC_AUC score: 0.95%


In [252]:
sample = vectorizer.transform(["bad hotel dirty washroom","good stay owner supportive"])
y_pred = xgb_model.predict(sample)
predictions = [round(value) for value in y_pred]
predictions

[0, 1]

In [423]:
sample = vectorizer.transform(resampled_df['lemmatized_comment'])
y_pred = xgb_model.predict(sample)
y_pred_prob = xgb_model.predict_proba(sample)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(resampled_df['Class'], predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(metrics.classification_report(resampled_df['Class'], y_pred))
print('ROC_AUC is {}'.format(metrics.roc_auc_score(resampled_df['Class'], y_pred)))
print(confusion_matrix(resampled_df['Class'], y_pred, labels=[0,1]))


Accuracy: 91.14%
              precision    recall  f1-score   support

           0       0.89      0.96      0.93      1150
           1       0.94      0.85      0.89       858

    accuracy                           0.91      2008
   macro avg       0.92      0.90      0.91      2008
weighted avg       0.91      0.91      0.91      2008

ROC_AUC is 0.903372859025033
[[1102   48]
 [ 130  728]]


In [294]:
# #RoBERTa

  
# import pandas as pd
# from simpletransformers.classification import ClassificationModel

# # Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
# train_df = train[['lemmatized_comment','Class']]

# eval_df = test[['lemmatized_comment','Class']]

# # Create a ClassificationModel
# model = ClassificationModel("roberta", "roberta-base", use_cuda=False)

# # Train the model
# model.train_model(train_df, use_cuda=False, output_dir='outputs2')

# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [424]:
resampled_df['prediction0'] = y_pred_prob[:,0]
resampled_df['prediction1'] = y_pred_prob[:,1]
resampled_df['prediction'] = predictions 

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [425]:
resampled_df

,Unnamed: 0,OYO ID,Hotel ID,OYO Product,Feedback ID,Booking ID,Source,Rating,L1,L2,...,Pricing,Default,Classification,comment(1cleaning),langid,lemmatized_comment,Class,prediction0,prediction1,prediction
0,1062,MRT042,46039,SMART,60144582,118672380,Android App,5,NaN,NaN,...,False,False,Good,i had a great stay hotel staff was very suppor...,en,great stay hotel staff supportive kind hearted...,1,0.003661,0.996339,1
1,283,KOL1340,82429,SMART,60156829,118141552,Android App,1,NaN,NaN,...,False,False,Bad,worst ever oyo experienced no oyo sign board m...,en,worst ever oyo experienced no oyo sign board m...,0,0.976968,0.023032,0
2,737,ASR293,58291,SMART,60168982,118586097,bulk_upload,0,NaN,NaN,...,False,True,Good,one of the best properties we have ever been i...,en,one best property ever would like visit strong...,1,0.188093,0.811907,1
3,1465,GRG1010,40843,SMART,60178229,118844212,Android App,5,NaN,NaN,...,False,False,Good,everything is good no complaints staff behavio...,en,everything good no complaint staff behaviour p...,1,0.020298,0.979702,1
4,654,DEL775,15322,SMART,60179150,118417540,Android App,5,NaN,NaN,...,False,False,Good,one of the best hotel through oyo i recommend ...,en,one best hotel oyo recommend hotel guest room ...,1,0.008773,0.991227,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383,905,DEL1525,55641,SPOT ON,61373230,118489815,Integrated OTA,2,NaN,NaN,...,False,True,Bad,refused to provide a discount for booking i ca...,en,refuse provide discount book say anything posi...,0,0.961890,0.038110,0
2384,989,GZB108,46852,SMART,61382951,121800300,IOS App,3,NaN,NaN,...,False,False,Bad,the curtains were very thin and incapable to k...,en,curtain thin incapable keep sunlight away cann...,0,0.988688,0.011312,0
2385,842,SRG102,56126,SMART,61390504,117518389,Android App,3,NaN,NaN,...,False,True,Bad,no local ids cards accepted and unmarried coup...,en,no local id card accept unmarried couple not w...,0,0.994857,0.005143,0
2386,2199,DEL1593,57984,Townhouse,61390678,122345895,Crs,1,NaN,NaN,...,False,False,Bad,wash room was not clean room was not clean pro...,en,wash room not clean room not clean properly co...,0,0.997769,0.002231,0


In [426]:
from sklearn.metrics import confusion_matrix
threshold = [0.9,0.8,0.7,0.6,0.5]
for t in threshold:
    print("Threshold: %.2f" % (t))
    threshold_df = resampled_df[(resampled_df.prediction0 > t) | (resampled_df.prediction1 > t)]
    tp = confusion_matrix(threshold_df['Class'], threshold_df['prediction'], labels=[0,1])[0][0]
    fn = confusion_matrix(threshold_df['Class'], threshold_df['prediction'], labels=[0,1])[1][0] + (resampled_df.shape[0] - threshold_df.shape[0])
    tn = confusion_matrix(threshold_df['Class'], threshold_df['prediction'], labels=[0,1])[0][1]
    fp = confusion_matrix(threshold_df['Class'], threshold_df['prediction'], labels=[0,1])[1][1]
    precision = float(tp) / (tp + tn)
    recall = float(tp) / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    print("Precision: %.2f" % (precision))
    print("Recall: %.2f" % (recall))
    print("F1 score: %.2f" % (f1))
    print("\n")

Threshold: 0.90
Precision: 0.99
Recall: 0.70
F1 score: 0.82


Threshold: 0.80
Precision: 0.98
Recall: 0.78
F1 score: 0.87


Threshold: 0.70
Precision: 0.98
Recall: 0.83
F1 score: 0.90


Threshold: 0.60
Precision: 0.97
Recall: 0.86
F1 score: 0.91


Threshold: 0.50
Precision: 0.96
Recall: 0.89
F1 score: 0.93


